In [ ]:
#referance
#https://python.langchain.com/docs/tutorials/agents/
#https://www.youtube.com/watch?v=yF9kGESAi3M&t=3823s

#agent
#llms just output texts , cannot perform actions
# agents -use llms to reason out their actions
#agent = (action->observation->thought)multiple iteration -> final answer
#tools
#a function schema - which define name, description and arguments
#tool calling
# a chat model that API , accepts tool schema
# we will be using Tavily a  search engine





In [3]:
#set the api keys
import os
from dotenv import load_dotenv

#from Basic_chat_model import messages


load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['TAVILY_API_KEY']=os.getenv("TAVILY_API_KEY")


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


In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search=TavilySearchResults(max_results=2)
search_result=search.invoke("What is the weather at Fremont now?")
#print(search_result)
tools=[search]


In [11]:
from langchain.chat_models import init_chat_model
model=init_chat_model("gpt-4o-mini", model_provider="openai")

In [12]:
from langchain_core.messages import HumanMessage
response=model.invoke([HumanMessage(content="Hi")])
response.content

'Hello! How can I assist you today?'

In [20]:
#bind the tools to the call
model_with_tools=model.bind_tools(tools)

In [21]:
# to understand the difference
#example 1
response=model_with_tools.invoke([HumanMessage(content="Hi")])
print("content :", response.content)
print("tool_calls :",response.tool_calls)

content : Hello! How can I assist you today?
tool_calls : []


In [22]:
#example 2
response=model_with_tools.invoke([HumanMessage(content="How is the weather at Fremont now?")])
print("content :", response.content)
print("tool_calls :",response.tool_calls)

content : 
tool_calls : [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather Fremont'}, 'id': 'call_vkxtkmCrIIoIU5gPvXM2ZB3A', 'type': 'tool_call'}]


In [23]:
# creating an agent
from langgraph.prebuilt import create_react_agent
agent_executor=create_react_agent(model,tools)



In [27]:
#simple hi message , no tools are invoked
response=agent_executor.invoke({"messages":[HumanMessage(content="Hi")]})
response["messages"]

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='203deea3-6770-40c3-a51a-03804e65400f'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 80, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BLDTSgMHPdakg7g4DDeKtNyFgXi5c', 'finish_reason': 'stop', 'logprobs': None}, id='run-13a5bcd1-f927-4c6b-b37c-77af4a2120c1-0', usage_metadata={'input_tokens': 80, 'output_tokens': 11, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [28]:
response=agent_executor.invoke({"messages":[HumanMessage(content="what is the weather at Fremont now?")]})
response["messages"]

[HumanMessage(content='what is the weather at Fremont now?', additional_kwargs={}, response_metadata={}, id='62f99bd5-eaa3-459f-83fb-8b6a1e4d015e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_90Br8EbUv5gvToZBUFXE98Wq', 'function': {'arguments': '{"query":"current weather Fremont"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 87, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BLDUlWm7cJNY6n2qrNy5GzB94kAXU', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c518de39-c915-44bb-8bfd-09f332e9c338-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'curren